# PySpark tutorial

## What is PySpark?


pyspark is a python api for working with apache spark.

* **Python api**: you can use the syntex and agility of python to interact with and send commands to a system that is not based, at its core, on python.

* **Apache Spark**: a system designed for working, analyzing and modeling with immense amounts of data in many computers at the same time. Putting it in a different way, apache spark allows you to run computations in parallel, instead of sequentially. It allows you to divide one incredibly large task into many smaller tasks, and run each such task on a different machine. This allowes you to accomplish your analysis goals in reasonable time that would not be possible on a single machine.

usually, we would define the amount of data that suits PySpark as what would not fit into a single machine storage (let alone RAM).

**important related concepts:**
1. distributed computing - when you distribute a task into several smaller task that run at the same time. this is what pyspark allows you to do with many machines, but it can also be done on a single machine with several threads, for example.
2. cluster - a network of machines that can take on tasks from a user, interact with one another and return results. these provide the computing resources that pyspark will use to make the computations.
3. Resilient Distributed Dataset (RDD) - an immutable distributed collection of data. it is not tabular and has no data schema. therefore, for tabular data wrangling, DataFrames allowes for more API options and uner-then-hood optimizations. still, you might encounter RDDs as you learn more about Spark, and should be aware of their existence.

**Part of PySpark we will cover:**
1. PySpark SQL - contains commands for data processing and manipulation.
2. PySpark MLlib - includes a variety of models, model training and related commands.

**Spark Architecture:**
to send commands and receive results from a cluster, you will need to initiate a spark session. this object is your tool for interacting with Spark. each user of the cluster will have its own Spark Session, that will allow him to use the cluster in isolation from other users. all of the sessions are communicating with a spark context, which is the master node in the cluster - that is, it assigns each of computers in the cluster tasks and coordinates them. each of the computers in the cluster that perform tasks for a master node is called a worker node. to connect to a worker node, the master node needs to get that node's comput power allocated to it, by a cluster manager, that is responsable for distributing the cluster resources. inside each worker node, there are execute programs that run the tasks - they can run multiple tasks simultaneously, and has their own cashe for storing results. so, each master node can have multiple worker nodes, that can have multiple tasks running.  

In [ ]:
!uname -a

In [ ]:
!pip install pyspark # ~ 1 min

In [ ]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
# !tar xf spark-3.0.2-bin-hadoop2.7.tgz
# !pip install -q findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

# Spark DataFrame Basics

Spark DataFrames allow for easy handling of large datasets.

* Easy syntax
* Ability to use SQL directly in the dataframe
* Operations are automatically distributed across RDDs

## Create a DataFrame


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("pyspark_basics").getOrCreate()

In [ ]:
%%writefile user_simple.json
{"name":"Bob"}
{"name":"Jim", "age":40}
{"name":"Mary", "age": 24}

In [ ]:
df = spark.read.json("user_simple.json")

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
df

## Show DataFrame


In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.describe().show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Specifying Schema Structure

- Some data types make it easier to infer schema.

- Often have to set the schema yourself

- Spark has tools to help specify the structure

Next we need to create the list of Structure fields
  * :param name: string, name of the field.
  * :param dataType: :class:`DataType` of the field.
  * :param nullable: boolean, whether the field can be null (None)

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
data_schema = [StructField("age", IntegerType(), True), StructField("name",StringType(), True)]

In [ ]:
final_struc = StructType(fields=data_schema)

In [ ]:
df = spark.read.json("user_simple.json", schema=final_struc)

In [ ]:
df.printSchema()

In [ ]:
df.show()

## Grab Data

In [ ]:
df['age']

In [ ]:
type(df['age'])

In [ ]:
df.select("age")

In [ ]:
type(df.select("age"))

In [ ]:
df.select("age").show()

In [ ]:
df.head(2)

In [ ]:
df.select(["name","age"])

In [ ]:
df.select(["name","age"]).show()

## Create New Columns

In [ ]:
df.withColumn("newAge", df['age']).show()

In [ ]:
df.show()

In [ ]:
df.withColumnRenamed("name","firstName").show()

In [ ]:
df.show()

In [ ]:
df.withColumn("agePlusTen", df['age']+10).show()

In [ ]:
df.withColumn("age_minus_5", df['age']-5).show()

## Using SQL

In [ ]:
df.createOrReplaceTempView("custmers")

In [ ]:
sql_results = spark.sql("SELECT * from custmers")

In [ ]:
sql_results

In [ ]:
sql_results.show()

In [ ]:
spark.sql("SELECT * FROM custmers WHERE age=24").show()

## DataFrame Operations

- Cover basic operations with Spark DataFrames.
- Use stock data from Walmart.

In [ ]:
!curl https://raw.githubusercontent.com/markumreed/colab_pyspark/main/WMT.csv >> WMT.csv

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("operations").getOrCreate()
df = spark.read.csv('WMT.csv',inferSchema=True,header=True)

In [ ]:
df.printSchema()

In [ ]:
df.head(5)

## Filtering Data

- DataFrames allow for quick filtering of data based on conditions


In [ ]:
df.filter('Close<62').show()

In [ ]:
df.filter('Close<62').select('Open').show()

In [ ]:
df.filter('Close<62').select(['Date','Open']).show()

## Using Comparison Operators
- Using comparison operators will look similar to SQL operators
- Make to call the entire column within the dataframe

In [ ]:
df.filter(df['Close'] < 62).show()

In [ ]:
df.filter((df['Close'] < 62) & ~(df['Open'] > 60)).show()

In [ ]:
df.filter(df['Open'] == 60.98).show(1)

In [ ]:
df.filter(df['Open'] == 60.98).collect()

In [ ]:
res =df.filter(df['Open'] == 60.98).collect()

In [ ]:
type(res[0])

In [ ]:
res[0].asDict()

In [ ]:
for item in res[0]:
  print(item)

In [ ]:
import pandas as pd

In [ ]:
pd.Series(res[0].asDict())

# GroupBy and Aggregate Functions
- `GroupBy` allows you to group rows together based off some column value
- Once you've performed the `GroupBy` operation you can use an aggregate function off that data.
- An aggregate function aggregates multiple rows of data into a single output, such as taking the sum of inputs, or counting the number of inputs.



In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("groupbyagg").getOrCreate()

## Import Data


In [ ]:
!curl https://raw.githubusercontent.com/markumreed/colab_pyspark/main/sales_data.csv >> sales_data.csv

In [ ]:
df = spark.read.csv("sales_data.csv", inferSchema=True, header=True)

In [ ]:
df.printSchema()

In [ ]:
df.show()

## Grouping Data
- Group the data by company

In [ ]:
df.groupBy("company")

## Aggregate Functions
- mean, count, max, min, sum...

In [ ]:
df.groupBy("company").mean().show()

In [ ]:
df.groupBy("company").count().show()

In [ ]:
df.groupBy("company").min().show()

In [ ]:
df.groupBy("company").max().show()

In [ ]:
df.groupBy("company").sum().show()

## Aggregating

- Not all methods need a groupby call, instead you can just call the generalized `.agg()` method, that will call the aggregate across all rows in the dataframe column specified.
- It can take in arguments as a single column, or create multiple aggregate calls all at once using dictionary notation.


In [ ]:
df.agg({"num_sales":"max"}).show()

In [ ]:
df.groupBy("company").agg({"num_sales":"mean"}).show()

In [ ]:
company_groups = df.groupBy("company")

In [ ]:
company_groups.min().show()

## Functions
There are a variety of functions you can import from pyspark.sql.functions.

In [ ]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [ ]:
df.select(countDistinct("num_sales")).show()

In [ ]:
df.select(avg("num_sales")).show()

In [ ]:
df.select(stddev("num_sales")).show()

### Alias
- To change the name, use the `.alias()` method for this:

In [ ]:
df.select(countDistinct("num_sales").alias("ANYTHING WE WANT")).show()

### Precision
- Use the `format_number` to change precision


In [ ]:
from pyspark.sql.functions import format_number

In [ ]:
sales_std = df.select(stddev("num_sales").alias("stddev"))

In [ ]:
sales_std.show()

In [ ]:
sales_std.select(format_number("stddev",2)).show()

## Order By


In [ ]:
df.orderBy("num_sales").show() # Ascending Order

In [ ]:
df.orderBy(df['num_sales'].desc()).show()

# Missing Data

- Often data sources are incomplete
- There are 3 options for filling in missing data:
  1. Just keep the missing data points.
  1. Drop them missing data points/row
  1. Fill them in with some other value.

## Keeping the missing data
A few machine learning algorithms can easily deal with missing data, let's see what it looks like:

In [ ]:
!curl https://raw.githubusercontent.com/markumreed/colab_pyspark/main/missing_data.csv >> missing_data.csv

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("missing_data").getOrCreate()

In [ ]:
df = spark.read.csv("missing_data.csv", header=True, inferSchema=True)

In [ ]:
df.show()

In [ ]:
df.printSchema()

## Drop the missing data

You can use the `.na` functions for missing data. The `drop` command has the following parameters:

```df.na.drop(how='any', thresh=None, subset=None)```
    
    * param how: 'any' or 'all'.
    
        If 'any', drop a row if it contains any nulls.
        If 'all', drop a row only if all its values are null.
    
    * param thresh: int, default None
    
        If specified, drop rows that have less than `thresh` non-null values.
        This overwrites the `how` parameter.
        
    * param subset:
        optional list of column names to consider.


In [ ]:
df.na.drop().show()

In [ ]:
df.na.drop(thresh=2).show()

In [ ]:
df.na.drop(subset=['sales']).show()

In [ ]:
df.na.drop(how='any').show()

In [ ]:
df.na.drop(how='all').show()

## Fill the missing values

We can also fill the missing values with new values. If you have multiple nulls across multiple data types, Spark is actually smart enough to match up the data types. For example:


In [ ]:
df.na.fill('سهیل').show()

In [ ]:
df.na.fill(999).show()

In [ ]:
df.na.fill("Missing Name", subset=["name"]).show()

In [ ]:
from pyspark.sql.functions import mean

In [ ]:
mean_value = df.select(mean(df['sales'])).collect()

In [ ]:
mean_sales_value = mean_value[0][0]

In [ ]:
df.na.fill(mean_sales_value, ["sales"]).show()

In [ ]:
# DON'T DO THIS
df.na.fill(df.select(mean(df['sales'])).collect()[0][0] ,['sales']).show() # NOT EASY TO READ

# Dates and Timestamps

You will often find yourself working with Time and Date information


In [ ]:
!curl https://raw.githubusercontent.com/markumreed/colab_pyspark/main/WMT.csv >> WMT.csv

In [ ]:
spark = SparkSession.builder.appName('walmart_dates').getOrCreate()

In [ ]:
df = spark.read.csv('WMT.csv', header=True, inferSchema=True)

In [ ]:
df.show()

In [ ]:
from pyspark.sql.functions import format_number, dayofmonth, hour, dayofyear, month, year, weekofyear, date_format

In [ ]:
df.select(dayofmonth(df['Date'])).show()

In [ ]:
df.select(hour(df['Date'])).show()

In [ ]:
df.select(dayofyear(df['Date'])).show()

In [ ]:
df.select(month(df['Date'])).show()

Find Avg Close Price per month.

In [ ]:
df.withColumn("Month", month(df['Date'])).show()

In [ ]:
df2 = df.withColumn("Month", month(df['Date']))

In [ ]:
df2.groupBy("Month").mean()[['avg(Month)', 'avg(Close)']].show()

In [ ]:
res = df2.groupBy("Month").mean()[['avg(Month)', 'avg(Close)']]
res = res.withColumnRenamed("avg(Month)", "Month")
res = res.select("Month", format_number('avg(Close)',2).alias("Mean Close")).show()

# Spark DataFrames Review

In [ ]:
!curl https://raw.githubusercontent.com/markumreed/colab_pyspark/main/appl_stock.csv >> apple_stock.csv

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("apple_stock").getOrCreate()

In [ ]:
df = spark.read.csv("apple_stock.csv", header=True, inferSchema=True)

In [ ]:
df.columns

In [ ]:
df.printSchema()

In [ ]:
df.head(5)

In [ ]:
df.describe().show()

In [ ]:
df.describe().printSchema()

In [ ]:
from pyspark.sql.functions import format_number

In [ ]:
res = df.describe()

In [ ]:
df.describe().columns

In [ ]:
res.select(res["summary"],
             format_number(res['Open'].cast('float'), 2).alias('Open'),
             format_number(res['High'].cast('float'), 2).alias('High'),
             format_number(res['Low'].cast('float'), 2).alias('Low'),
             format_number(res['Close'].cast('float'), 2).alias('Close'),
             res['Volume'] .cast('int').alias('Volume')
             ).show()

In [ ]:
# High vs Volume
df2 = df.withColumn("HV Ratio", df['High']/df['Volume'])

In [ ]:
df2.show()

In [ ]:
df2.select('HV Ratio').show()

In [ ]:
df.orderBy(df['High'].desc()).head(1)[0][0]

In [ ]:
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

In [ ]:
from pyspark.sql.functions import max, min

In [ ]:
df.select(max('Volume'), min('Volume')).show()

In [ ]:
df.filter("Close < 120").count()

In [ ]:
from pyspark.sql.functions import count

In [ ]:
res = df.filter('Close < 1210')
res.select(count('Close')).show()

In [ ]:
(df.filter('High > 180').count() * 1.0/df.count()) * 100

In [ ]:
from pyspark.sql.functions import corr

In [ ]:
df.select(corr('High', 'Volume')).show()

In [ ]:
from pyspark.sql.functions import year
yeardf = df.withColumn("Year", year(df['Date']))

In [ ]:
max_df = yeardf.groupBy('Year').max()

In [ ]:
max_df.select('Year', 'max(High)').show()

In [ ]:
max_df.show()

In [ ]:
from pyspark.sql.functions import month

In [ ]:
monthdf = df.withColumn("Month", month("Date"))
monthavgs = monthdf.select("Month", "Close").groupBy("Month").mean()
monthavgs.select("Month", "avg(Close)").orderBy('Month').show()

# Linear Regression with PySpark

- Based on the Official Spark Documentation for PySpark

In [ ]:
!curl https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_linear_regression_data.txt >> sample_linear_regression_data.txt

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("lr_ex").getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [ ]:
training.show()

In [ ]:
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [ ]:
lrModel = lr.fit(training)

In [ ]:
print("Coefficients:", str(lrModel.coefficients))
print("Intercept:", str(lrModel.intercept))

In [ ]:
trainSummary = lrModel.summary

In [ ]:
print("MAE: ", trainSummary.meanAbsoluteError)
print("MSE: ", trainSummary.meanSquaredError)
print("RMSE: ", trainSummary.rootMeanSquaredError)
print("R2: ", trainSummary.r2)
print("Adj R2: ", trainSummary.r2adj)


## Train Test Split with PySpark
- Pass in the split between training/test as a list.
-  No correct, but generally 70/30 or 60/40 splits are used.
-  Depending on how much data you have and how unbalanced it is.

In [ ]:
df = spark.read.format("libsvm").load("sample_linear_regression_data.txt") # FULL DATASET

In [ ]:
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)

In [ ]:
test_data.show()

In [ ]:
unlabeled_data = test_data.select('features')

In [ ]:
corrected_model = lr.fit(train_data)

In [ ]:
res = corrected_model.evaluate(test_data)

In [ ]:
print("MAE: ", res.meanAbsoluteError)
print("MSE: ", res.meanSquaredError)
print("RMSE: ", res.rootMeanSquaredError)
print("R2: ", res.r2)
print("Adj R2: ", res.r2adj)

In [ ]:
predictions = corrected_model.transform(unlabeled_data)

In [ ]:
predictions.show()

# Data Transformations with PySpark

## Data Features
### StringIndexer
- Convert string data into numerical (categorical feature)
- Encode as dummy variables/OneHotEncoder
- `StringIndexer`

In [ ]:
from pyspark.ml.feature import StringIndexer

df2 = spark.createDataFrame(
    [(0,"a"), (1, "b"), (2, "c"), (3, "a"), (4, "b"), (5, "c")],
    ["user_id", "category"]
)

In [ ]:
df2.show()

+-------+--------+
|user_id|category|
+-------+--------+
|      0|       a|
|      1|       b|
|      2|       c|
|      3|       a|
|      4|       b|
|      5|       c|
+-------+--------+



In [ ]:
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")

In [ ]:
indexed = indexer.fit(df2).transform(df2)

In [ ]:
indexed.show()

+-------+--------+-------------+
|user_id|category|categoryIndex|
+-------+--------+-------------+
|      0|       a|          0.0|
|      1|       b|          1.0|
|      2|       c|          2.0|
|      3|       a|          0.0|
|      4|       b|          1.0|
|      5|       c|          2.0|
+-------+--------+-------------+



## VectorIndexer
- **VectorAssembler** is a transformer that combines a given list of columns into a single vector column.
- **VectorAssembler** accepts the following input column types:
  - all numeric types, boolean type, and vector type.  

---

- Assume that we have a DataFrame with the columns id, hour, mobile, userFeatures, and clicked:

id | hour | mobile | userFeatures     | clicked
----|------|--------|------------------|---------
0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0
     
- userFeatures is a vector column that contains three user features.  
- After transformation we should get the following DataFrame:

id | hour | mobile | userFeatures     | clicked | features
----|------|--------|------------------|---------|-----------------------------
0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0     | [18.0, 1.0, 0.0, 10.0, 0.5]

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
df3 = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"]
)
df3.show()

+---+----+------+--------------+-------+
| id|hour|mobile|  userFeatures|clicked|
+---+----+------+--------------+-------+
|  0|  18|   1.0|[0.0,10.0,0.5]|    1.0|
+---+----+------+--------------+-------+



In [ ]:
assembler = VectorAssembler(
    inputCols = ["hour", "mobile", "userFeatures"],
    outputCol = "features"
)
output = assembler.transform(df3)

In [ ]:
output.select("features", "clicked").show()

+--------------------+-------+
|            features|clicked|
+--------------------+-------+
|[18.0,1.0,0.0,10....|    1.0|
+--------------------+-------+



## PySpark - Linear Regression (Another example)

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("lin_reg").getOrCreate()

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/PySpark/ecommerce.csv", inferSchema=True, header=True)

In [ ]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
df.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [ ]:
df.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [ ]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App',
                                       'Time on Website','Length of Membership'],
                            outputCol='features')

In [ ]:
output = assembler.transform(df)

In [ ]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [ ]:
final_data = output.select("features", "Yearly Amount Spent")

In [ ]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [ ]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                364|
|   mean|  502.9869765747741|
| stddev|  78.20081373178272|
|    min| 298.76200786180766|
|    max|  765.5184619388373|
+-------+-------------------+



In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                136|
|   mean|  489.4835268829236|
| stddev|  81.70383291336739|
|    min| 256.67058229005585|
|    max|  669.9871405017029|
+-------+-------------------+



In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [ ]:
model = lr.fit(train_data)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame({"Coefficients":model.coefficients}, index=['Avg Session Length', 'Time on App',
                                       'Time on Website','Length of Membership'])

,Coefficients
Avg Session Length,25.528189
Time on App,38.739248
Time on Website,0.381506
Length of Membership,61.116148


In [ ]:
res = model.evaluate(test_data)

In [ ]:
res.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|   9.669149274977144|
| -13.253743124870539|
|  -7.014907673268283|
|  -18.93401211450589|
|   3.666788132691522|
| -13.506343132392146|
| -7.6506770546077405|
|   8.515390472010267|
|  18.287686613130802|
|  3.2878745843414663|
|-0.09345137585660268|
| -5.8919286856935855|
|  -6.045290947261833|
| -17.994114323187887|
| -14.543665761366128|
| -2.7649679910356895|
|   6.936300111304206|
| -10.178724246025183|
|0.017993930797047142|
|   7.475084221903728|
+--------------------+
only showing top 20 rows



In [ ]:
unlabeled_data = test_data.select("features")

In [ ]:
predictions = model.transform(unlabeled_data)

In [ ]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|398.97120179765034|
|[30.3931845423455...|333.18261292806415|
|[30.4925366965402...| 289.4861533931828|
|[30.8162006488763...| 285.0203530629749|
|[31.0472221394875...| 388.8306110563299|
|[31.0662181616375...| 462.4396363400665|
|[31.1280900496166...| 564.9033638016624|
|[31.1695067987115...|418.84114033028254|
|[31.3123495994443...| 445.3037314148098|
|[31.3584771924370...|491.88807586513394|
|[31.3895854806643...| 410.1630624358395|
|[31.5171218025062...| 281.8103493360793|
|[31.5257524169682...| 450.0109177571437|
|[31.5702008293202...| 563.9396064645928|
|[31.5741380228732...|  558.952937921953|
|[31.8186165667690...|449.18364136117134|
|[31.8209982016720...|417.73898090190914|
|[31.8279790554652...| 450.1814717929667|
|[31.8293464559211...|385.13434405717794|
|[31.8512531286083...|465.51716244489467|
+--------------------+------------

In [ ]:
print("MAE:", res.meanAbsoluteError)
print("MSE:", res.meanSquaredError)
print("RMSE:", res.rootMeanSquaredError)
print("R2", res.r2)
print("Adj R2", res.r2adj)

MAE: 7.7682724106361
MSE: 92.46118689887506
RMSE: 9.615674022078487
R2 0.9860466096979039
Adj R2 0.985620551978756


# Logistic Regression with PySpark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("log_reg").getOrCreate()

In [ ]:
!curl https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt >> sample_libsvm_data_2.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  102k  100  102k    0     0   467k      0 --:--:-- --:--:-- --:--:--  467k


In [ ]:
df = spark.read.format("libsvm").load("sample_libsvm_data_2.txt")

In [ ]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression()

model = lr.fit(df)

summary = model.summary

In [ ]:
summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514862...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198867...|[6.76550380001560...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678715831...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012719...|[4.62137287298722...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874697...|[1.81823629113437...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504187...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212813...|[6.97903542824686...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503566...|[3.00582577441380...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606570...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [ ]:
model.evaluate(df)

In [ ]:
pred_and_labels = model.evaluate(df)

In [ ]:
pred_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514862...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198867...|[6.76550380001560...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678715831...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012719...|[4.62137287298722...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874697...|[1.81823629113437...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504187...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212813...|[6.97903542824686...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503566...|[3.00582577441380...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606570...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [ ]:
pred_and_labels = pred_and_labels.predictions.select("label", "prediction")

In [ ]:
pred_and_labels.show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



# Evaluation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [ ]:
eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")

In [ ]:
eval_multi = MulticlassClassificationEvaluator(predictionCol="prediction",
                                               labelCol="label",
                                               metricName="accuracy")

In [ ]:
acc = eval.evaluate(pred_and_labels)

In [ ]:
acc

1.0

# Logistic Regression: Titantic Dataset

In [ ]:
!curl https://raw.githubusercontent.com/markumreed/colab_pyspark/main/titanic.csv >> titanic.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   378k      0 --:--:-- --:--:-- --:--:--  379k


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("titanic").getOrCreate()

In [ ]:
df = spark.read.csv("titanic.csv", inferSchema=True, header=True)

In [ ]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [ ]:
data = df.select([
 'Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [ ]:
data.head()

Row(Survived=0, Pclass=3, Sex='male', Age=22.0, SibSp=1, Parch=0, Fare=7.25, Embarked='S')

In [ ]:
data_final = data.na.drop()

# Categorical Data with PySpark

In [ ]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,
                                OneHotEncoder, StringIndexer)

In [ ]:
gender_indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
gender_ecoder = OneHotEncoder(inputCol="SexIndex", outputCol="SexVec")

embark_indexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkIndex")
embark_ecoder = OneHotEncoder(inputCol="EmbarkIndex", outputCol="EmbarkVec")


In [ ]:
assembler = VectorAssembler(inputCols=["Pclass", "SexVec", "Age", "SibSp",
                                       "Parch", "Fare", "EmbarkVec"],
                            outputCol="features")

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Pipelines

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
lr = LogisticRegression(featuresCol='features', labelCol="Survived")

In [ ]:
pipeline = Pipeline(stages=[
                            gender_indexer,embark_indexer,
                            gender_ecoder,embark_ecoder,
                            assembler, lr
])

In [ ]:
train, test = data_final.randomSplit([0.7, 0.3], seed=42)

In [ ]:
model_fit = pipeline.fit(train)
res = model_fit.transform(test)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                     labelCol='Survived')

In [ ]:
res.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [ ]:
auc = eval.evaluate(res)

In [ ]:
auc

0.7747561675272518

# Clustering with PySpark


In [ ]:
!curl https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_kmeans_data.txt >> sample_kmeans_data.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   120  100   120    0     0    863      0 --:--:-- --:--:-- --:--:--   869


# K-means


In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sample_cluster").getOrCreate()

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [ ]:
df = spark.read.format("libsvm").load("sample_kmeans_data.txt")

In [ ]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [ ]:
kmeans = KMeans().setK(2).setSeed(42)
model = kmeans.fit(df)

In [ ]:
pred = model.transform(df)

In [ ]:
eval = ClusteringEvaluator()

In [ ]:
silhouette = eval.evaluate(pred)
print(f"Silhouette with squared euclidean distance: {silhouette}")

Silhouette with squared euclidean distance: 0.9997530305375207


In [ ]:
centers = model.clusterCenters()
print("Cluster Centers:")
print("=================")
for center in centers:
  print(center)

Cluster Centers:
[0.1 0.1 0.1]
[9.1 9.1 9.1]


In [ ]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt >> seeds_dataset.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9300    0  9300    0     0  28174      0 --:--:-- --:--:-- --:--:-- 28267


# Random Forest Classifier with PySpark

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("rf").getOrCreate()

In [ ]:
df = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [ ]:
df.show()

# Train Test Split

In [ ]:
(train, test) = df.randomSplit([0.7, 0.3], seed=42)

In [ ]:
test.show()

In [ ]:
train.printSchema()

# Train RF Model

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20,seed=42)

In [ ]:
model = rf.fit(train)

In [ ]:
pred = model.transform(test)

In [ ]:
pred.printSchema()

In [ ]:
pred.select("prediction", "label", "features").show(5)

In [ ]:
eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [ ]:
acc = eval.evaluate(pred)

In [ ]:
print("Test Error = %g" % (1.0 - acc))

In [ ]:
model.featureImportances

# Gradient Boosted Trees

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10, seed=42)

In [ ]:
model = gbt.fit(train)

In [ ]:
pred = model.transform(test)

In [ ]:
pred.select("prediction", "label", "features").show(5)

In [ ]:
eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acc = eval.evaluate(pred)
print("Test Error = %g" % (1.0 - acc))

## Tree Methods with PySpark
1. Single Decision Tree
1. Random Forest
1. Gradient Boosted Tree Classifier

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("trees").getOrCreate()

In [ ]:
df = spark.read.csv("College.csv", inferSchema=True, header=True)

In [ ]:
df.printSchema()

In [ ]:
df.head(2)

In [ ]:
df.columns

# Formatting for Spark

In [ ]:
# "label", "features"
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
assembler = VectorAssembler(
    inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'
    ],
    outputCol="features"
)

In [ ]:
output = assembler.transform(df)

# String Variables (Private)

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndexer")
output_fixed = indexer.fit(output).transform(output)

In [ ]:
df_final = output_fixed.select("features", "PrivateIndexer")

In [ ]:
train, test = df_final.randomSplit([0.7, 0.3], seed=42)

# Tree Classifiers

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml import Pipeline

## CREATE Models

In [ ]:
dtc = DecisionTreeClassifier(labelCol="PrivateIndexer", featuresCol="features")
rfc = RandomForestClassifier(labelCol="PrivateIndexer", featuresCol="features")
gbt = GBTClassifier(labelCol="PrivateIndexer", featuresCol="features")

In [ ]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

# Predictions

In [ ]:
dtc_pred = dtc_model.transform(test)
rfc_pred = rfc_model.transform(test)
gbt_pred = gbt_model.transform(test)

# Eval

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndexer", predictionCol="prediction", metricName="accuracy")

In [ ]:
dtc_acc = evaluator.evaluate(dtc_pred)
rfc_acc = evaluator.evaluate(rfc_pred)
gbt_acc = evaluator.evaluate(gbt_pred)

In [ ]:
print("-"*10)
print(f"DT Acc: {dtc_acc}")
print("-"*10)
print(f"RFC Acc: {rfc_acc}")
print("-"*10)
print(f"GBT Acc: {gbt_acc}")
print("-"*10)